# **Scraping**

# DETIK HEALTH

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from datetime import datetime, timedelta
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

def get_article_content(link):
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')
        div_content = soup.find('div', class_='detail__body-text')
        if not div_content:
            return "", ""

        paragraphs = div_content.find_all('p')
        content = ' '.join([p.get_text(strip=True) for p in paragraphs])

        description = content.split(".")[0] + "." if content else ""
        return content, description
    except Exception as e:
        print(f"Error artikel: {link} | {e}")
        return "", ""

def parse_date(time_info):
    try:
        # Format dengan hari & tanggal (contoh: "Senin, 15 Sep 2025 08:30 WIB")
        if any(bulan in time_info for bulan in
               ["Jan", "Feb", "Mar", "Apr", "Mei", "Jun", "Jul", "Agu", "Sep", "Okt", "Nov", "Des"]):
            time_info = time_info.replace("WIB", "").strip()
            dt = datetime.strptime(time_info, "%A, %d %b %Y %H:%M")
            return dt.strftime("%Y-%m-%d %H:%M:%S")

        # Format "X jam yang lalu"
        match = re.match(r"(\d+)\s+jam yang lalu", time_info)
        if match:
            hours = int(match.group(1))
            dt = datetime.now() - timedelta(hours=hours)
            return dt.strftime("%Y-%m-%d %H:%M:%S")

        # Format "X menit yang lalu"
        match = re.match(r"(\d+)\s+menit yang lalu", time_info)
        if match:
            minutes = int(match.group(1))
            dt = datetime.now() - timedelta(minutes=minutes)
            return dt.strftime("%Y-%m-%d %H:%M:%S")

        # Format "X hari yang lalu"
        match = re.match(r"(\d+)\s+hari yang lalu", time_info)
        if match:
            days = int(match.group(1))
            dt = datetime.now() - timedelta(days=days)
            return dt.strftime("%Y-%m-%d %H:%M:%S")

        return time_info
    except Exception as e:
        print(f"Error parsing date: {time_info} | {e}")
        return time_info

total_scraped = 0
max_articles = 100
page = 1

with open('detik_health.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['title', 'tag', 'link', 'date', 'description', 'content'])

    while total_scraped < max_articles:
        url = f'https://health.detik.com/berita-detikhealth?p={page}'
        print(f"\n📄 Scraping halaman {page}...")

        try:
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('article')

            if not articles:
                print("Tidak ada artikel lagi di halaman ini. Menghentikan proses.")
                break

            for art in articles:
                if total_scraped >= max_articles:
                    break

                try:
                    title_element = art.find('h3', class_='media__title')
                    if not title_element:
                        continue

                    a_tag = title_element.find('a')
                    title = title_element.text.strip()
                    link = a_tag['href']
                    time_info_raw = art.find('div', class_='media__date').text.strip()
                    time_info = parse_date(time_info_raw)

                    content, description = get_article_content(link)

                    writer.writerow([title, "", link, time_info, description, content])

                    print(f"✅ Scraping ke-{total_scraped + 1}: {title}")
                    total_scraped += 1
                    time.sleep(1)

                except Exception as e:
                    print(f"Error parsing artikel di halaman {page}: {e}")
                    continue

            page += 1
            time.sleep(1)

        except Exception as e:
            print(f"Gagal akses page {page}: {e}")
            break

print(f"\n🎉 Proses scraping selesai! Total artikel: {total_scraped}")



📄 Scraping halaman 1...
✅ Scraping ke-1: Strategi BPOM RI Berantas Produk Obat dan Makanan Mengandung Bahan Berbahaya
✅ Scraping ke-2: Curhat Pria Lahir dari Donor Sperma, Bingung Cari Ayah Kandung
✅ Scraping ke-3: Pastikan Indomie Aman, Indofood Soroti Beda Regulasi RI Vs Taiwan soal Etilen Oksida
✅ Scraping ke-4: Ilmuwan Jepang Temukan Cara Hapus Ingatan, Harapan Baru Mengobati Trauma
✅ Scraping ke-5: Mengenal Golongan Darah Paling Langka di Dunia, Ada yang Dijuluki 'Golden Blood Type'
✅ Scraping ke-6: BPOM RI Simpulkan Tak Ada Pelanggaran pada Kasus Indomie Soto Banjar di Taiwan
✅ Scraping ke-7: Menkes Sebut 'NutriGrade' di RI Tinggal Tunggu Waktu, Targetnya Tahun Ini
✅ Scraping ke-8: Penjelasan Indofood soal Indomie Soto Banjar Kuit Tersandung Temuan Etilen Oksida
✅ Scraping ke-9: BPOM Sudah Kantongi Hasil Uji Ompreng MBG Diduga Mengandung Minyak Babi
✅ Scraping ke-10: Rahasia Panjang Umur Shigeko Kagawa, Wanita Tertua di Jepang Berusia 114 Tahun
✅ Scraping ke-11: 4 Manfaat Minum 

In [ ]:
from google.colab import files
files.download("detik_health.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
f = pd.read_csv('detik_health.csv', delimiter=';')
pd.set_option("display.max_colwidth", 150)
display(f.head(20))

,title,tag,link,date,description,content
0,Strategi BPOM RI Berantas Produk Obat dan Makanan Mengandung Bahan Berbahaya,NaN,https://health.detik.com/berita-detikhealth/d-8114838/strategi-bpom-ri-berantas-produk-obat-dan-makanan-mengandung-bahan-berbahaya,2025-09-16 08:45:51,Badan Pengawas Obat dan Makanan (BPOM RI) bergerak cepat untuk memberantas produk obat dan makanan mengandung bahan berbahaya.,Badan Pengawas Obat dan Makanan (BPOM RI) bergerak cepat untuk memberantas produk obat dan makanan mengandung bahan berbahaya. BPOM berkomitmen un...
1,"Curhat Pria Lahir dari Donor Sperma, Bingung Cari Ayah Kandung",NaN,https://health.detik.com/berita-detikhealth/d-8114689/curhat-pria-lahir-dari-donor-sperma-bingung-cari-ayah-kandung,2025-09-16 07:45:53,Seorang pria di Inggris merupakan salah satu anak yang lahir dari donor sperma.,Seorang pria di Inggris merupakan salah satu anak yang lahir dari donor sperma. Ia dikandung melalui program bayi tabung atau In Vitro Fertilizati...
2,"Pastikan Indomie Aman, Indofood Soroti Beda Regulasi RI Vs Taiwan soal Etilen Oksida",NaN,https://health.detik.com/berita-detikhealth/d-8114504/pastikan-indomie-aman-indofood-soroti-beda-regulasi-ri-vs-taiwan-soal-etilen-oksida,2025-09-16 06:45:55,Laporan kandungan etilen oksida (EtO) dalam Indomie varian Soto Banjar Limau Kuit belakangan memicu kekhawatiran publik.,Laporan kandungan etilen oksida (EtO) dalam Indomie varian Soto Banjar Limau Kuit belakangan memicu kekhawatiran publik. Temuan tersebut dilaporka...
3,"Ilmuwan Jepang Temukan Cara Hapus Ingatan, Harapan Baru Mengobati Trauma",NaN,https://health.detik.com/berita-detikhealth/d-8114224/ilmuwan-jepang-temukan-cara-hapus-ingatan-harapan-baru-mengobati-trauma,2025-09-16 04:45:58,"Otak memiliki kemampuan luar biasa untuk menyimpan informasi sebagai ingatan, memungkinkan kita belajar dari kesalahan.","Otak memiliki kemampuan luar biasa untuk menyimpan informasi sebagai ingatan, memungkinkan kita belajar dari kesalahan. Namun, tidak semua ingatan..."
4,"Mengenal Golongan Darah Paling Langka di Dunia, Ada yang Dijuluki 'Golden Blood Type'",NaN,https://health.detik.com/berita-detikhealth/d-8113591/mengenal-golongan-darah-paling-langka-di-dunia-ada-yang-dijuluki-golden-blood-type,2025-09-16 03:46:00,"Golongan darah emas (Golden blood type), atau Rh null, merupakan golongan darah paling langka di dunia karena sama sekali tidak memiliki antigen R...","Golongan darah emas (Golden blood type), atau Rh null, merupakan golongan darah paling langka di dunia karena sama sekali tidak memiliki antigen R..."
5,BPOM RI Simpulkan Tak Ada Pelanggaran pada Kasus Indomie Soto Banjar di Taiwan,NaN,https://health.detik.com/berita-detikhealth/d-8114137/bpom-ri-simpulkan-tak-ada-pelanggaran-pada-kasus-indomie-soto-banjar-di-taiwan,2025-09-16 03:46:04,Kepala Badan Pengawas Obat dan Makanan (BPOM RI) Taruna Ikrar menyimpulkan tidak menemukan adanya pelanggaran pada kasus mi instan 'Indomie Soto B...,Kepala Badan Pengawas Obat dan Makanan (BPOM RI) Taruna Ikrar menyimpulkan tidak menemukan adanya pelanggaran pada kasus mi instan 'Indomie Soto B...
6,"Menkes Sebut 'NutriGrade' di RI Tinggal Tunggu Waktu, Targetnya Tahun Ini",NaN,https://health.detik.com/berita-detikhealth/d-8114094/menkes-sebut-nutrigrade-di-ri-tinggal-tunggu-waktu-targetnya-tahun-ini,2025-09-16 02:46:06,"Menteri Kesehatan RI Budi Gunadi Sadikin memastikan penetapan labeling seperti yang dilakukan Singapura pada minuman dengan kadar gula tinggi, aka...","Menteri Kesehatan RI Budi Gunadi Sadikin memastikan penetapan labeling seperti yang dilakukan Singapura pada minuman dengan kadar gula tinggi, aka..."
7,Penjelasan Indofood soal Indomie Soto Banjar Kuit Tersandung Temuan Etilen Oksida,NaN,https://health.detik.com/berita-detikhealth/d-8114055/penjelasan-indofood-soal-indomie-soto-banjar-kuit-tersandung-temuan-etilen-oksida,2025-09-16 02:46:09,Gaduh Indomie varian Soto Banjar Limau Kuit dilaporkan teridentifikasi etilen oksida (EtO) di luar ambang batas aman yan

# Kompas Health

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

def get_article_content_and_desc(link):
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')

        # ambil isi artikel
        div_content = soup.find('div', class_='read__content')
        content = ''
        if div_content:
            paragraphs = div_content.find_all('p')
            content = ' '.join([p.get_text(strip=True) for p in paragraphs])

        # ambil meta description
        meta_desc = ''
        meta_tag = soup.find('meta', attrs={'name': 'description'})
        if meta_tag and meta_tag.get('content'):
            meta_desc = meta_tag['content'].strip()

        # tentukan description
        if meta_desc:
            description = meta_desc
        else:
            sentences = re.split(r'(?<=[.!?])\s+', content.strip())
            description = sentences[0] if sentences else ''

        return content, description

    except Exception as e:
        print(f"Error artikel: {link} | {e}")
        return '', ''


with open('kompas_health.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['title', 'tag', 'link', 'date', 'description', 'content'])

    for page in range(1, 4):  # contoh 3 halaman pertama
        print(f"\n📄 Scraping halaman {page}...")
        url = f'https://health.kompas.com/?page={page}'
        try:
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('div', class_='trenLatest__item')

            for art in articles:
                try:
                    a_tag = art.find('h3', class_='trenLatest__title').find('a', class_='tren__link')
                    title = a_tag.text.strip()
                    link = a_tag['href']
                    tag = art.find('div', class_='tren__subtitle').text.strip()
                    date_info = art.find('div', class_='tren__date').text.strip()

                    content, description = get_article_content_and_desc(link)

                    writer.writerow([title, tag, link, date_info, description, content])
                    print(f"✅ {title}")
                    time.sleep(1)
                except Exception as e:
                    print(f"Error parsing artikel di halaman {page}: {e}")
                    continue

            time.sleep(1)
        except Exception as e:
            print(f"Gagal akses page {page}: {e}")
            continue

print("\n🎉 Proses scraping selesai!")



📄 Scraping halaman 1...
✅ 1. Cara Sederhana Mengatasi Brain Fog
✅ 2. Pemprov Babel Segera Bangun Rumah Sakit Jantung dan Stroke
✅ 3. Kusta Jadi Masalah Serius di Papua Barat, Kemenkes Diminta Bentuk Tim Investigasi Penanganan Kasus
✅ 4. Modul Penanganan Psikosomatis Dokter Indonesia Dipresentasikan di Jerman
✅ 5. Anak Obesitas Kini Lebih Banyak daripada Anak Kurang Gizi
✅ 6. Gejala Diabetes Tipe 1 pada Anak, Banyak Minum hingga Berat Badan Turun
✅ 7. Lebihi Batas Aman, Luruhan BPA di Galon Guna Ulang Bahayakan Kesehatan
✅ 8. Penyebab Makin Banyak Orang Muda Sakit Nyeri Lutut
✅ 9. Jangan Bagikan Data Medis ke AI, Ini Risiko yang Mengintai
✅ 10. Diabetes Tipe 1 pada Anak, Pentingnya Diagnosis Dini untuk Cegah Kematian
✅ 11. Bencana Banjir di Bali, Waspadai Bermacam Penyakit yang Dapat Muncul
✅ 12. Era Baru Pengobatan Kanker Darah Multiple Myeloma
✅ 13. Tak Ada Gejala Khas, Kanker Darah Multiple Myeloma Selalu Terlambat Dideteksi
✅ 14. Bahaya Heatstroke akibat Kurang Cairan dan Cuaca Pan

In [ ]:
from google.colab import files
files.download("kompas_health.csv")

In [ ]:
import pandas as pd
f = pd.read_csv('kompas_health.csv', delimiter=';')
pd.set_option("display.max_colwidth", 150)
display(f.head(20))

,title,tag,link,date,description,content
0,Cara Sederhana Mengatasi Brain Fog,HEALTH,https://health.kompas.com/read/25I15144420968/cara-sederhana-mengatasi-brain-fog,"15/09/2025, 14:44 WIB","Brain fog merujuk pada sekumpulan gejala kognitif seperti kesulitan berkonsentrasi, mudah lupa, dan kelambatan berpikir.","KOMPAS.com -Ketika kesibukan dan tingkat stres meningkat, bukan hal yang aneh jika kita tiba-tiba masuk ke ruangan dan lupa tujuan, lupa istilah s..."
1,Pemprov Babel Segera Bangun Rumah Sakit Jantung dan Stroke,HEALTH,https://health.kompas.com/read/25I14224517668/pemprov-babel-segera-bangun-rumah-sakit-jantung-dan-stroke,"14/09/2025, 22:45 WIB","Pemprov Kepulauan Babel rencanakan pembangunan RS Jantung dan Stroke dekat Bandara, untuk meningkatkan layanan kesehatan masyarakat.",KOMPAS.com- Pembangunan Rumah Sakit Jantung dan Stroke di Provinsi Kepulauan Bangka Belitung (Babel) bertujuan memperkuat infrastruktur layanan ke...
2,"Kusta Jadi Masalah Serius di Papua Barat, Kemenkes Diminta Bentuk Tim Investigasi Penanganan Kasus",HEALTH,https://health.kompas.com/read/25I14222312268/kusta-jadi-masalah-serius-di-papua-barat-kemenkes-diminta-bentuk-tim-investigasi,"14/09/2025, 22:23 WIB","DPD RI desak Kemenkes bentuk tim investigasi kusta di Papua Barat. Penanganan masih terkendala obat, tenaga medis, dan tingginya risiko penularan.","KOMPAS.com- Kasus kusta di Papua Barat masih menjadi persoalan serius. Hingga tahun 2024, tercatat 796 orang penderita tersebar di enam kabupaten,..."
3,Modul Penanganan Psikosomatis Dokter Indonesia Dipresentasikan di Jerman,HEALTH,https://health.kompas.com/read/25I13184500268/modul-penanganan-psikosomatis-dokter-indonesia-dipresentasikan-di-jerman,"13/09/2025, 18:45 WIB","Sejak 2010, dr. Andri memfokuskan praktik klinisnya pada pasien dengan gangguan psikosomatis dan kecemasan.",KOMPAS.com -Dalam ajang bergengsi European Academy of Psychosomatic Medicine (EAPM) Annual Congress 2025 yang berlangsung pada 11–13 September di...
4,Anak Obesitas Kini Lebih Banyak daripada Anak Kurang Gizi,HEALTH,https://health.kompas.com/read/25I13080000068/anak-obesitas-kini-lebih-banyak-daripada-anak-kurang-gizi,"13/09/2025, 08:00 WIB",Data UNICEF menunjukkan tren yang cukup mengkhawatirkan. Jumlah anak yang mengalami obesitas terus meningkat di dunia.,"KOMPAS.com- Untuk pertama kalinya dalam sejarah, jumlah anak yang mengalami obesitas di dunia melampaui mereka yang kekurangan berat badan. Temua..."
5,"Gejala Diabetes Tipe 1 pada Anak, Banyak Minum hingga Berat Badan Turun",HEALTH,https://health.kompas.com/read/25I12181500168/gejala-diabetes-tipe-1-pada-anak-banyak-minum-hingga-berat-badan-turun,"12/09/2025, 18:15 WIB","Kenali gejala diabetes tipe 1 pada anak seperti sering kencing dan berat badan turun. Deteksi dini bisa menyelamatkan buah hati, menurut dokter.","KOMPAS.com -Diabetes tipe 1 pada anak masih sering terlambat dikenali, padahal gejalanya bisa terlihat dari kebiasaan sehari-hari. Jika orangtua l..."
6,"Lebihi Batas Aman, Luruhan BPA di Galon Guna Ulang Bahayakan Kesehatan",HEALTH,https://health.kompas.com/read/25I12153053268/lebihi-batas-aman-luruhan-bpa-di-galon-guna-ulang-bahayakan-kesehatan,"12/09/2025, 15:30 WIB","Paparan BPA dapat mengganggu hormon, reproduksi, hingga jantung. Pakar minta pengawasan dan pelabelan ketat.","KOMPAS.com –Paparan bahan kimia berbahaya Bisfenol A (BPA) secara terus-menerus dalam jangka panjang dapat berdampak serius bagi kesehatan, teruta..."
7,Penyebab Makin Banyak Orang Muda Sakit Nyeri Lutut,HEALTH,https://health.kompas.com/read/25I12122026868/penyebab-makin-banyak-orang-muda-sakit-nyeri-lutut,"12/09/2025, 12:20 WIB",Kerusakan pada sendi lutut seringkali tidak menunjukkan gejala dan sudah bisa dimulai sejak dini.,"KOMPAS.com -Selama ini, nyeri lutut identik dengan proses penuaan. Namun, tren terkini justru menunjukkan fenomena yang mengkhawatirkan, semakin ..."
8,"Jangan Bagikan Data Medis ke AI, Ini Risiko yang Mengintai",HEALTH,https://health.kompas.com/read/25I